This notebook shows how to calculate all the angles. There are three major functions for the calculation. The <code>filter_sensor_points_to_cube_id</code> function returns only the sensor points that corresponds to one HSI cube. This significantly increases the efficiency and make the process faster. The second function <code>get_closest_sensor_point</code> returns a csv file that attaches the information from the closest sensor points for each raster pixel. The angles are calculated outside the functions. However, they can be converted into a function to form a loop for a number of HSI cubes.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Transformer
from shapely.geometry import Point
from scipy.spatial import distance
from pvlib import solarposition
from tqdm.notebook import tqdm

In [ ]:
# Define the working directory
os.chdir('YOUR WORKING DIRECTORY')

In [ ]:
def filter_sensor_points_to_cube_id(sensor_filename, raster_filename):
    
    """
    This function filters out only the sensor coordinates that align with
    the cube boundary. Basically it relies on corresponding timestamp for
    the cubes which is stored in a file "frameindex_cubeid.txt". The 
    corresponding UTC time of the timestamp is store id the "imu_gps.txt"
    file. Based on that, only the in between coordinates are selected and
    returned.
    
    Input:
        - sensor_filename: (str) path of the sensor filename as ASCII format
        - raster_filename: (str) path of the raster filename as csv format
        
        
    """
    
    #######################################################################
    # Read the sensor coordinates
    sensor = pd.read_csv(sensor_filename, sep="\"", header=None)
    # Rename the columns
    sensor.columns = ['Time', 'Lat_v', 'Lon_v']
    
    # Insert new columns for X and Y
    sensor.insert(3, "X_v", 0)
    sensor.insert(4, "Y_v", 0)

    # Convert lat lon to X and Y in UTM 15N
    transformer = Transformer.from_crs(4326, 32615)
    Xv, Yv = transformer.transform(sensor.iloc[:, 1], sensor.iloc[:, 2])
    sensor.loc[:, 'X_v'] = Xv
    sensor.loc[:, 'Y_v'] = Yv
    
    # Convert the time string to a timestamp column
    sensor['Time_UTC'] = pd.to_datetime(sensor['Time'])
    # Drop the string time column
    sensor.drop(columns=['Time'], inplace=True)
    #######################################################################
    
    
    #######################################################################
    # Get the cubeid from the raster_filename
    cube_id = os.path.basename(raster_filename).split('_')[1]
    # Generate the frame filename
    frame_filename = os.path.join(os.path.dirname(raster_filename),
                                  f'frameIndex_{cube_id}.txt')
    # Generate the imu+gps filename
    imu_filename = os.path.join(os.path.dirname(raster_filename),
                                'imu_gps.txt')
    
    # Read frame and imu_gps files as df
    frame = pd.read_csv(frame_filename, sep="\t", header=0)
    imu = pd.read_csv(imu_filename, sep="\t", header=0,
                      parse_dates=['Gps_UTC_Date&Time'])
    #######################################################################
    
    
    #######################################################################
    # Get the starting and ending frame timestamp
    start_frame = frame.iloc[0, -1]
    end_frame = frame.iloc[-1, -1]
    
    # Get the closest starting timestamp date
    start_imu = pd.DatetimeIndex(imu.iloc[(imu['Timestamp']-start_frame).abs().argsort()[:1], 7])
    # Add a 20s offset
    start_imu = start_imu - pd.to_timedelta(20, unit='s')
    # Get the string time information
    start_imu = start_imu.strftime('%Y-%M-%d %H:%M:%-S')[0]

    # Get the closest starting timestamp date
    end_imu = pd.DatetimeIndex(imu.iloc[(imu['Timestamp']-end_frame).abs().argsort()[:1], 7])
    # Add a 16s offset
    end_imu = end_imu - pd.to_timedelta(16, unit='s')
    # Get the string time information
    end_imu = end_imu.strftime('%Y-%M-%d %H:%M:%-S')[0]
    
    # Filter the sensor df
    sensor_filter = sensor[(sensor['Time_UTC'] >= start_imu) & (sensor['Time_UTC'] <= end_imu)]
    #######################################################################
    
    return sensor_filter

In [ ]:
def get_closest_sensor_point(raster_filename, sensor_filename):

    # Read the raster point csv file
    raster = pd.read_csv(raster_filename, index_col=0)

    # Split the rasters into 4 different parts
    raster1, raster2, raster3, raster4 = np.array_split(raster, 4)
    # Delete the raster
    del raster

    # Read the sensor shapefile
    sensor = filter_sensor_points_to_cube_id(sensor_filename, raster_filename)
    # Take only the X, Y
    #sensor = sensor[['X_m', 'Y_m', 'Z']]
    # Give the observations a new id
    sensor['sensor_id'] = np.arange(0, sensor.shape[0])

    # Create an empty list to hold the processed dfs
    raster_sensor = []
    count = 0
    
    # Loop through every df
    for raster_split in [raster1, raster2, raster3, raster4]:

        # Get the X and Y from each dataframe
        R = raster_split[['X_r', 'Y_r']].values
        V = sensor[['X_v', 'Y_v']].values

        # Calcualte the distance
        dist = distance.cdist(R, V, 'euclidean')
        # Calculate the minimum distance index
        argmin_dist = np.argmin(dist, axis=1)

        # Add the minimum sensor index to raster
        raster_split['sensor_id'] = argmin_dist

        # Join the sensor information to the raster split
        raster_split_sensor = raster_split.join(sensor.set_index('sensor_id'), on='sensor_id')

        # Add the df to the list
        raster_sensor.append(raster_split_sensor)
        
        print(f"Part {count+1} done")
        
        count = count + 1
        
    # Create a pandas dataframe from the list of dfs
    raster_sensor = pd.concat(raster_sensor)
    # Drop the sensor_id
    raster_sensor.drop(columns=['sensor_id'], inplace=True)
    
    return raster_sensor

In [ ]:
# Define the filenames
sensor_filename = "./Data/imu_gps.txt"
raster_filename = "./Data/raw_0_rd_rf_or_pr_warp.csv"

Get the closest sensor points in a dataframe.

In [ ]:
%%time
raster_sensor = get_closest_sensor_point(raster_filename, sensor_filename)

In [ ]:
# View the dataframe
raster_sensor

Function to calculate SZA (Solar Zenith Angle)

In [ ]:
def calculate_sensor_zenith_angle(R, V):
    
    return 90-np.degrees(np.arctan(50.0 / np.linalg.norm(R-V, axis=1)))

In [ ]:
# Add the sza into the dataframe
raster_sensor['VZA'] = calculate_sensor_zenith_angle(raster_sensor.iloc[:, 0:2].values,
                                                     raster_sensor.iloc[:, 6:8].values)

In [ ]:
# Get the datetime index and localize it
datetime_idx = pd.DatetimeIndex(raster_sensor['Time_UTC']).tz_convert('America/Chicago')
# The local timezone should be changed based on the location

# Equation of time
equation_of_time = solarposition.equation_of_time_spencer71(datetime_idx.dayofyear).values

# Hour angle in degrees
ha = solarposition.hour_angle(datetime_idx,
                              raster_sensor['Lon_r'].values,
                              equation_of_time)

# Solar declination in radians
declination = solarposition.declination_cooper69(datetime_idx.dayofyear).values

# Solar zenith angle in radians
sza = solarposition.solar_zenith_analytical(np.radians(raster_sensor['Lat_r']),
                                            np.radians(ha),
                                            declination).values

# Solar azimuth angle in radians
saa = solarposition.solar_azimuth_analytical(np.radians(raster_sensor['Lat_r']),
                                             np.radians(ha), declination, sza)

In [ ]:
# Add the SZA and SAA to the dataframe, convert it to degrees
raster_sensor['SZA'] = np.degrees(sza)
raster_sensor['SAA'] = np.degrees(saa)

Function to calculate VAA (Viewing Azimuth Angle)

In [ ]:
def calculate_viewing_azimuth_angle(X_v, Y_v, X_r, Y_r):

    V = np.array([X_v, Y_v])
    R = np.array([X_r, Y_r])

    a = np.array([  0., 100.])
    b = V - R
    unit_a = a/np.linalg.norm(a)
    unit_b = b/np.linalg.norm(b)
    dot_prod = np.dot(unit_a, unit_b)

    return np.degrees(np.arccos(np.clip(dot_prod, -1.0, 1.0)))

Apply the VAA function to each row in the dataframe. Lambda function method was found to be the most efficient way.

In [ ]:
%%time
raster_sensor.loc[:, 'VAA'] = raster_sensor.apply(lambda row: calculate_viewing_azimuth_angle(row['X_v'],
                                                         row['Y_v'],
                                                         row['X_r'],
                                                         row['Y_r'],), axis=1)

In [ ]:
# Check the dataframe
raster_sensor

Convert the csv rows in to a ESRI shapefile.

In [ ]:
def convert_to_shape(df, X, Y, out_path):
    geometry = [Point(xy) for xy in zip(df[X], df[Y])]
    point_gdf = gpd.GeoDataFrame(df, crs="EPSG:32615", geometry=geometry)
    point_gdf.to_file(out_path)

In [ ]:
convert_to_shape(raster_sensor.drop(columns=['Time_UTC']), 'X_r', 'Y_r', r"F:\danforthstudy\temp\angle_test.shp")

The shapefile was interpolated using ArcGIS Natural Neighbor (https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/natural-neighbor.htm) tool. While doing the interpolation, 0.1 m was considered as the raster resolution. The outputs for the given datasaet are added in the <code>Outputs</code> directory.